In [2]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()
tracker.stop()

tracker.start()

[codecarbon WARNING @ 20:40:36] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 20:40:36] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon INFO @ 20:40:36] [setup] RAM Tracking...
[codecarbon INFO @ 20:40:36] [setup] CPU Tracking...
[codecarbon WARNING @ 20:40:40] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 20:40:40] CPU Model on constant consumption mode: Apple M1
[codecarbon WARNING @ 20:40:40] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 20:40:40] [setup] GPU Tracking...
[codecarbon INFO @ 20:40:40] No GPU found.
[codecarbon INFO @ 20:40:40] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant


In [3]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score

from kan import KAN


In [4]:
df = pd.read_csv("../training.csv")

df.head()

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


In [5]:
# Reemplazar valores inválidos
df.replace(-999.0, np.nan, inplace=True)

# Separar features y target
X = df.drop(columns=['EventId', 'Weight', 'Label'])
y = df['Label'].map({'s': 1, 'b': 0})  # Convertir a 1 y 0

In [6]:
# Imputar con la media
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
X = pd.DataFrame(X_imputed, columns=X.columns)

In [7]:
# Normalizar entre 0 y 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (200000, 30), Test shape: (50000, 30)


In [9]:
import torch

# Convertir a tensores
train_inputs = torch.tensor(X_train, dtype=torch.float32)
test_inputs = torch.tensor(X_test, dtype=torch.float32)
train_labels = torch.tensor(y_train.to_numpy(), dtype=torch.long)
test_labels = torch.tensor(y_test.to_numpy(), dtype=torch.long)

# Crear dataset para el modelo
dataset = {
    'train_input': train_inputs,
    'train_label': train_labels,
    'test_input': test_inputs,
    'test_label': test_labels
}


In [10]:
def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).float())

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).float())

In [11]:
import numpy as np
import torch

# Calcular pesos inversamente proporcionales a la frecuencia de clase
y_counts = np.bincount(dataset['train_label'].numpy())
total = y_counts.sum()
class_weights = total / (2 * y_counts)  # normalizados

# Convertir a tensor
weights_tensor = torch.tensor(class_weights, dtype=torch.float32)


In [12]:
model = KAN(
    width=[24, 24, 16, 8, 2],
    grid=7,
    k=3,
    seed=0,
    device='cpu'
)

results = model.fit(
    dataset,
    opt="Adam",
    metrics=(train_acc, test_acc),
    loss_fn=torch.nn.CrossEntropyLoss(weight=weights_tensor),
    steps=200,
    lamb=0.00001,
    lamb_entropy=0.001
)


checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                          | 0/200 [00:00<?, ?it/s][codecarbon INFO @ 20:40:59] Energy consumed for RAM : 0.000042 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 20:40:59] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 20:40:59] Energy consumed for All CPU : 0.000021 kWh
[codecarbon INFO @ 20:40:59] 0.000063 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:41:14] Energy consumed for RAM : 0.000083 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 20:41:14] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 20:41:14] Energy consumed for All CPU : 0.000042 kWh
[codecarbon INFO @ 20:41:14] 0.000125 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:41:29] Energy consumed for RAM : 0.000124 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 20:41:29] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO 

saving model version 0.1


In [13]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score
import torch

with torch.no_grad():
    logits = model(dataset['test_input'])
    y_pred_test = torch.argmax(logits, dim=1).numpy()
    y_true_test = dataset['test_label'].numpy()
    y_prob_test = torch.softmax(logits, dim=1)[:, 1].numpy()

acc = accuracy_score(y_true_test, y_pred_test)
recall = recall_score(y_true_test, y_pred_test)
f1 = f1_score(y_true_test, y_pred_test)
auc = roc_auc_score(y_true_test, y_prob_test)

print("🔍 Métricas finales del modelo KAN")
print(f"Accuracy:  {acc:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"AUC-ROC:   {auc:.4f}")


🔍 Métricas finales del modelo KAN
Accuracy:  0.8136
Recall:    0.7593
F1 Score:  0.7363
AUC-ROC:   0.8838


In [14]:
# Prune del modelo entrenado
model_pruned = model.prune(node_th=1e-4, edge_th=1e-4)


saving model version 0.2


In [15]:
with torch.no_grad():
    logits_pruned = model_pruned(dataset['test_input'])
    y_pred_pruned = torch.argmax(logits_pruned, dim=1).numpy()
    y_prob_pruned = torch.softmax(logits_pruned, dim=1)[:, 1].numpy()

acc_p = accuracy_score(y_true_test, y_pred_pruned)
recall_p = recall_score(y_true_test, y_pred_pruned)
f1_p = f1_score(y_true_test, y_pred_pruned)
auc_p = roc_auc_score(y_true_test, y_prob_pruned)

print("✂️ Métricas del modelo KAN pruneado:")
print(f"Accuracy:  {acc_p:.4f}")
print(f"Recall:    {recall_p:.4f}")
print(f"F1 Score:  {f1_p:.4f}")
print(f"AUC-ROC:   {auc_p:.4f}")


✂️ Métricas del modelo KAN pruneado:
Accuracy:  0.8115
Recall:    0.7630
F1 Score:  0.7350
AUC-ROC:   0.8832


In [16]:
emissions = tracker.stop()

[codecarbon INFO @ 01:11:28] Energy consumed for RAM : 0.030196 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:11:28] Delta energy consumed for CPU with constant : 0.000018 kWh, power : 5.0 W
[codecarbon INFO @ 01:11:28] Energy consumed for All CPU : 0.015124 kWh
[codecarbon INFO @ 01:11:28] 0.045321 kWh of electricity used since the beginning.
